<a href="https://colab.research.google.com/github/Charitha-05/Real_Time_Violence_Detection_System/blob/main/Copy_of_Copy_of_Final_Violence_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls '/content/drive/My Drive/'

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from transformers import TimesformerModel, TimesformerConfig
from transformers import VideoMAEModel, VideoMAEConfig
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install opencv-python

In [ ]:
# Import transformer libraries
from transformers import (
    VideoMAEForVideoClassification,
    VideoMAEImageProcessor,
    TimesformerForVideoClassification,
    AutoImageProcessor,
    TrainingArguments,
    Trainer
)

In [ ]:
# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
VIDEO_DIR = '/content/drive/MyDrive/Real Life Violence Dataset'  # path to video dataset
IMG_SIZE = 224                            # height & width of frames
NUM_FRAMES = 16                           # number of frames to sample per video
SR = 16000                                # audio sampling rate (Hz)
DURATION = 3                              # seconds of audio snippet to extract
N_MELS = 128                               # mel-spectrogram bins for audio

In [ ]:
import os

class_names = sorted(os.listdir(VIDEO_DIR))
print("Classes:", class_names)

for cls in class_names:
    cls_dir = os.path.join(VIDEO_DIR, cls)
    num_videos = len([f for f in os.listdir(cls_dir) if f.endswith(('.mp4', '.avi', '.mov'))])
    print(f"Class '{cls}' has {num_videos} videos")


In [ ]:
import os

total_size = 0

for cls in os.listdir(VIDEO_DIR):
    cls_dir = os.path.join(VIDEO_DIR, cls)
    for f in os.listdir(cls_dir):
        if f.endswith(('.mp4', '.avi', '.mov')):
            file_path = os.path.join(cls_dir, f)
            size_mb = os.path.getsize(file_path) / (1024 * 1024)  # bytes → MB
            total_size += size_mb
            print(f"{f} : {size_mb:.2f} MB")

print(f"\nTotal dataset size: {total_size:.2f} MB")


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

OUTPUT_DIR = '/content/violence_dataset_split'

classes = ['NonViolence','Violence']

# Create folder structure
for split in ['train','test']:
    for cls in classes:
        os.makedirs(os.path.join(OUTPUT_DIR, split, cls), exist_ok=True)

# Collect all video paths and labels
video_paths, labels = [], []
for idx, cls in enumerate(classes):
    cls_dir = os.path.join(VIDEO_DIR, cls)
    for f in os.listdir(cls_dir):
        if f.endswith(('.mp4','.avi','.mov')):
            video_paths.append(os.path.join(cls_dir, f))
            labels.append(idx)

# Split 80/20
train_paths, test_paths, train_labels, test_labels = train_test_split(
    video_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Copy files to new folders
for path, label in zip(train_paths, train_labels):
    cls = classes[label]
    shutil.copy(path, os.path.join(OUTPUT_DIR, 'train', cls))

for path, label in zip(test_paths, test_labels):
    cls = classes[label]
    shutil.copy(path, os.path.join(OUTPUT_DIR, 'test', cls))

print("✅ Dataset split into train/test folders completed.")


In [ ]:
from sklearn.model_selection import train_test_split
import os
import pandas as pd

# Dataset directoryife Violence Dataset'

# Define class names
classes = ['NonViolence', 'Violence']

# Collect video paths and labels
video_paths, labels = [], []
for idx, cls in enumerate(classes):
    cls_dir = os.path.join(VIDEO_DIR, cls)
    for f in os.listdir(cls_dir):
        if f.lower().endswith(('.mp4', '.avi', '.mov')):
            video_paths.append(os.path.join(cls_dir, f))
            labels.append(idx)

# Split dataset (80% train, 20% test)
train_paths, test_paths, train_labels, test_labels = train_test_split(
    video_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Convert to DataFrames for easier viewing
train_df = pd.DataFrame({'path': train_paths, 'label': train_labels})
test_df = pd.DataFrame({'path': test_paths, 'label': test_labels})

# Map numeric labels to class names
train_df['class_name'] = train_df['label'].map({i: c for i, c in enumerate(classes)})
test_df['class_name'] = test_df['label'].map({i: c for i, c in enumerate(classes)})

# Print summary
print("="*60)
print("✅ Dataset Split Summary")
print("="*60)
print(f"Total Samples: {len(video_paths)}")
print(f"Train Samples: {len(train_paths)}")
print(f"Test Samples:  {len(test_paths)}\n")

print("📊 Class Distribution:")
print("Train set:")
print(train_df['class_name'].value_counts())
print("\nTest set:")
print(test_df['class_name'].value_counts())

In [ ]:
import base64
from IPython.display import HTML, display
import random
import pandas as pd

# --- Helper: Convert a video file to HTML5 playable format (Base64) ---
def video_to_html(path, width=400):
    try:
        with open(path, "rb") as f:
            data = f.read()
        b64 = base64.b64encode(data).decode('utf-8')
        return f'<video width="{width}" controls><source src="data:video/mp4;base64,{b64}" type="video/mp4"></video>'
    except Exception as e:
        return f"<p style='color:red;'>Error loading video: {e}</p>"

# --- Helper: Pick one random video path for a given class ---
def pick_video(df, classname):
    subset = df[df['class_name'] == classname]
    if len(subset) == 0:
        return None
    return random.choice(subset['path'].tolist())

# --- Pick sample videos from each class/split ---
train_violence_path = pick_video(train_df, 'Violence')
train_nonviolence_path = pick_video(train_df, 'NonViolence')
test_violence_path = pick_video(test_df, 'Violence')
test_nonviolence_path = pick_video(test_df, 'NonViolence')

# --- Convert each to HTML5 playable Base64 ---
train_violence_html = video_to_html(train_violence_path)
train_nonviolence_html = video_to_html(train_nonviolence_path)
test_violence_html = video_to_html(test_violence_path)
test_nonviolence_html = video_to_html(test_nonviolence_path)

# --- Build final HTML grid layout ---
html_content = f"""
<table style="width:100%; text-align:center;">
  <tr>
    <td>{train_violence_html}<br><b>Train - Violence</b></td>
    <td>{train_nonviolence_html}<br><b>Train - NonViolence</b></td>
  </tr>
  <tr>
    <td>{test_violence_html}<br><b>Test - Violence</b></td>
    <td>{test_nonviolence_html}<br><b>Test - NonViolence</b></td>
  </tr>
</table>
"""

# --- Display in notebook ---
display(HTML(html_content))

# --- Print selected paths for clarity ---
print("🎬 Selected video paths:")
print(f"Train - Violence:     {train_violence_path}")
print(f"Train - NonViolence:  {train_nonviolence_path}")
print(f"Test - Violence:      {test_violence_path}")
print(f"Test - NonViolence:   {test_nonviolence_path}")


In [ ]:
!pip install torch torchvision torchaudio
!pip install decord

In [ ]:
import torch
from torch.utils.data import Dataset
import decord
from decord import VideoReader, cpu
import torchvision.transforms as T

decord.bridge.set_bridge('torch')  # output videos as torch tensors

class VideoDataset(Dataset):
    def __init__(self, video_paths, labels, num_frames=16, frame_size=(224,224)):
        self.video_paths = video_paths
        self.labels = labels
        self.num_frames = num_frames
        self.frame_size = frame_size

        # Transform: Resize + Normalize
        self.transform = T.Compose([
            T.Resize(frame_size),  # resize each frame
            T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        path = self.video_paths[idx]
        label = self.labels[idx]

        vr = VideoReader(path, ctx=cpu(0))
        total_frames = len(vr)

        # Sample num_frames evenly across the video
        frame_indices = torch.linspace(0, total_frames-1, self.num_frames).long()
        frames = vr.get_batch(frame_indices)  # (num_frames, H, W, C), dtype=torch.uint8

        # Convert to float and scale to [0,1]
        frames = frames.float() / 255.0

        # Permute to (num_frames, C, H, W)
        frames = frames.permute(0, 3, 1, 2)

        # Apply transform to each frame
        frames = torch.stack([self.transform(frame) for frame in frames])

        return frames, label


In [ ]:
from torch.utils.data import DataLoader

train_dataset = VideoDataset(train_paths, train_labels, num_frames=16, frame_size=(224,224))
test_dataset = VideoDataset(test_paths, test_labels, num_frames=16, frame_size=(224,224))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Check shapes
for videos, labels in train_loader:
    print(videos.shape, labels)
    break


In [ ]:
!pip install transformers timm


In [ ]:
from transformers import VideoMAEForVideoClassification, VideoMAEFeatureExtractor
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Number of classes
num_classes = 2  # Violence / NonViolence

# Load pretrained VideoMAE
video_model = VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
    num_labels=num_classes
)
video_model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(video_model.parameters(), lr=1e-4)


In [ ]:
epochs = 5  # adjust as needed

for epoch in range(epochs):
    video_model.train()
    running_loss = 0.0

    for videos, labels in train_loader:
        videos = videos.to(device)          # (B, F, C, H, W)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = video_model(videos).logits     # forward pass
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")


In [ ]:
video_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for videos, labels in test_loader:
        videos = videos.to(device)
        labels = labels.to(device)
        outputs = video_model(videos).logits
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")

In [ ]:
# === SAVE VIDEO MODEL ===
video_model_save_dir = "/content/drive/MyDrive/video_violence_detection_model"

video_model.save_pretrained(video_model_save_dir)
print(f"✅ Video model saved successfully at: {video_model_save_dir}")


In [ ]:
import torch

# Pick a sample video (e.g., first in test set)
sample_video_path = test_paths[0]
sample_label = test_labels[0]

# Create a single-sample dataset and loader
sample_dataset = VideoDataset([sample_video_path], [sample_label], num_frames=16, frame_size=(224,224))
sample_loader = torch.utils.data.DataLoader(sample_dataset, batch_size=1, shuffle=False)

# Put model in evaluation mode
video_model.eval()

with torch.no_grad():
    for video, label in sample_loader:
        video = video.to(device)          # (1, F, C, H, W)
        label = label.to(device)

        outputs = video_model(video).logits
        _, predicted = torch.max(outputs, 1)

        print(f"True label: {label.item()}, Predicted: {predicted.item()}")


In [ ]:
import torch

# Provide the video path directly
sample_video_path = "/content/violence_dataset_split/test/NonViolence/NV_120.mp4"  # replace with your actual path
# Provide the true label manually (0=NonViolence, 1=Violence)
sample_label = 0

# Create a single-sample dataset and loader
sample_dataset = VideoDataset([sample_video_path], [sample_label], num_frames=16, frame_size=(224,224))
sample_loader = torch.utils.data.DataLoader(sample_dataset, batch_size=1, shuffle=False)

# Put model in evaluation mode
video_model.eval()

with torch.no_grad():
    for video, label in sample_loader:
        video = video.to(device)          # (1, F, C, H, W)
        label = label.to(device)

        outputs = video_model(video).logits
        _, predicted = torch.max(outputs, 1)

        print(f"Video Path: {sample_video_path}")
        print(f"True label: {label.item()} ({classes[label.item()]}), Predicted: {predicted.item()} ({classes[predicted.item()]})")


In [ ]:
import torch

# Provide the video path directly
sample_video_path = "/content/violence_dataset_split/test/Violence/V_131.mp4"  # replace with your actual path
# Provide the true label manually (0=NonViolence, 1=Violence)
sample_label = 1

# Create a single-sample dataset and loader
sample_dataset = VideoDataset([sample_video_path], [sample_label], num_frames=16, frame_size=(224,224))
sample_loader = torch.utils.data.DataLoader(sample_dataset, batch_size=1, shuffle=False)

# Put model in evaluation mode
video_model.eval()

with torch.no_grad():
    for video, label in sample_loader:
        video = video.to(device)          # (1, F, C, H, W)
        label = label.to(device)

        outputs = video_model(video).logits
        _, predicted = torch.max(outputs, 1)

        print(f"Video Path: {sample_video_path}")
        print(f"True label: {label.item()} ({classes[label.item()]}), Predicted: {predicted.item()} ({classes[predicted.item()]})")


In [ ]:
import torch
import cv2
from IPython.display import display, HTML
from base64 import b64encode
import numpy as np

# Provide the video path directly
sample_video_path = "/content/violence_dataset_split/test/Violence/V_14.mp4"  # replace with your actual path
sample_label = 1  # 0=NonViolence, 1=Violence

# Create a single-sample dataset and loader
sample_dataset = VideoDataset([sample_video_path], [sample_label], num_frames=16, frame_size=(224,224))
sample_loader = torch.utils.data.DataLoader(sample_dataset, batch_size=1, shuffle=False)

# Put model in evaluation mode
video_model.eval()

with torch.no_grad():
    for video, label in sample_loader:
        video = video.to(device)          # (1, F, C, H, W)
        label = label.to(device)

        outputs = video_model(video).logits
        _, predicted = torch.max(outputs, 1)

        print(f"Video Path: {sample_video_path}")
        print(f"True label: {label.item()} ({classes[label.item()]}), Predicted: {predicted.item()} ({classes[predicted.item()]})")

# --- Display the video in Colab ---
def show_video(path):
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""
        <video width=480 controls>
            <source src="{data_url}" type="video/mp4">
        </video>
    """))

show_video(sample_video_path)


In [ ]:
import torch
import cv2
from IPython.display import display, HTML
from base64 import b64encode
import numpy as np

# Provide the video path directly
sample_video_path = "/content/violence_dataset_split/test/NonViolence/NV_19.mp4"  # replace with your actual path
sample_label = 0  # 0=NonViolence, 1=Violence

# Create a single-sample dataset and loader
sample_dataset = VideoDataset([sample_video_path], [sample_label], num_frames=16, frame_size=(224,224))
sample_loader = torch.utils.data.DataLoader(sample_dataset, batch_size=1, shuffle=False)

# Put model in evaluation mode
video_model.eval()

with torch.no_grad():
    for video, label in sample_loader:
        video = video.to(device)          # (1, F, C, H, W)
        label = label.to(device)

        outputs = video_model(video).logits
        _, predicted = torch.max(outputs, 1)

        print(f"Video Path: {sample_video_path}")
        print(f"True label: {label.item()} ({classes[label.item()]}), Predicted: {predicted.item()} ({classes[predicted.item()]})")

# --- Display the video in Colab ---
def show_video(path):
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""
        <video width=480 controls>
            <source src="{data_url}" type="video/mp4">
        </video>
    """))

show_video(sample_video_path)


In [ ]:
import torch
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

video_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for videos, labels in test_loader:
        videos = videos.to(device)
        labels = labels.to(device)

        outputs = video_model(videos).logits
        _, predicted = torch.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

# Visualize
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
import os
from pydub import AudioSegment

# === Input: folder containing audio files ===
audio_dir = "/content/drive/MyDrive/archive (11)/audios_VSD/audios_VSD"  # change to your folder path

# Supported audio formats
valid_extensions = ('.mp3', '.wav', '.ogg', '.flac', '.m4a')

# List to store results
audio_durations = []

# Loop through all files in the directory
for filename in os.listdir(audio_dir):
    if filename.lower().endswith(valid_extensions):
        file_path = os.path.join(audio_dir, filename)
        try:
            audio = AudioSegment.from_file(file_path)
            duration = len(audio) / 1000.0  # milliseconds → seconds
            audio_durations.append((filename, duration))
            print(f"{filename}: {duration:.2f} seconds")
        except Exception as e:
            print(f"⚠️ Could not read {filename}: {e}")

# Optionally save results to CSV
import pandas as pd
df = pd.DataFrame(audio_durations, columns=["Filename", "Duration_seconds"])
output_csv = "/content/audio_durations.csv"
df.to_csv(output_csv, index=False)

print(f"\n✅ Durations saved to: {output_csv}")


In [ ]:
import os
import pandas as pd
import librosa
import soundfile as sf
import numpy as np

# === CONFIG ===
EXCEL_PATH = "/content/drive/MyDrive/audio dataset/VSD.xlsx"
AUDIO_DIR = "/content/drive/MyDrive/audio dataset/audios_VSD/audios_VSD"
OUTPUT_DIR = "/content/drive/MyDrive/split_audios"

os.makedirs(os.path.join(OUTPUT_DIR, "Violence"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "NonViolence"), exist_ok=True)

# === LOAD EXCEL ===
df = pd.read_excel(EXCEL_PATH)

# Clean column names
df.columns = [c.strip().lower() for c in df.columns]

# Expecting columns: file_segment_name, violence_start, violence_end
if not all(col in df.columns for col in ['file_segment_name', 'violence_start', 'violence_end']):
    raise ValueError("Excel must contain columns: File_segment_name, Violence_start, Violence_end")

# Group by each audio file
for file_name, group in df.groupby('file_segment_name'):
    # Add file extension (assuming .wav)
    audio_path = os.path.join(AUDIO_DIR, f"{file_name}.wav")
    if not os.path.exists(audio_path):
        print(f"⚠️ Missing audio file: {audio_path}")
        continue

    # Load full audio
    y, sr = librosa.load(audio_path, sr=16000)
    duration = librosa.get_duration(y=y, sr=sr)

    # Collect violent segments
    violent_ranges = [
        (max(0, float(row.violence_start)), min(duration, float(row.violence_end)))
        for _, row in group.iterrows()
        if not np.isnan(row.violence_start) and not np.isnan(row.violence_end)
    ]
    violent_ranges.sort(key=lambda x: x[0])

    # === Save violent segments ===
    for i, (start, end) in enumerate(violent_ranges):
        seg = y[int(start * sr): int(end * sr)]
        out_path = os.path.join(OUTPUT_DIR, "Violence", f"{file_name}_V{i}.wav")
        sf.write(out_path, seg, sr)

    # === Find non-violent ranges ===
    non_violent_ranges = []
    prev_end = 0.0
    for start, end in violent_ranges:
        if start - prev_end > 1.0:  # ignore short gaps <1s
            non_violent_ranges.append((prev_end, start))
        prev_end = end
    if duration - prev_end > 1.0:
        non_violent_ranges.append((prev_end, duration))

    # === Save non-violent segments ===
    for i, (start, end) in enumerate(non_violent_ranges):
        seg = y[int(start * sr): int(end * sr)]
        out_path = os.path.join(OUTPUT_DIR, "NonViolence", f"{file_name}_NV{i}.wav")
        sf.write(out_path, seg, sr)

    print(f"✅ Processed {file_name}: {len(violent_ranges)} violent, {len(non_violent_ranges)} non-violent segments")

print("\n🎯 All files processed successfully!")

In [ ]:
import glob
import pandas as pd
import os

OUTPUT_DIR = "/content/drive/MyDrive/split_audios"

violence = [(f, 1) for f in glob.glob(os.path.join(OUTPUT_DIR, "Violence", "*.wav"))]
nonviolence = [(f, 0) for f in glob.glob(os.path.join(OUTPUT_DIR, "NonViolence", "*.wav"))]

df = pd.DataFrame(violence + nonviolence, columns=["filepath", "label"])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df.to_csv(os.path.join(OUTPUT_DIR, "dataset.csv"), index=False)

print("✅ Created dataset.csv with", len(df), "samples")

In [ ]:
import os
import random
from IPython.display import Audio, display

# === CONFIG ===
VIOLENT_DIR = "/content/drive/MyDrive/split_audios/Violence"       # <-- change to your actual folder
NON_VIOLENT_DIR = "/content/drive/MyDrive/split_audios/NonViolence" # <-- change to your actual folder

# === GET RANDOM FILES ===
violent_files = [f for f in os.listdir(VIOLENT_DIR) if f.endswith(('.wav', '.mp3', '.ogg'))]
non_violent_files = [f for f in os.listdir(NON_VIOLENT_DIR) if f.endswith(('.wav', '.mp3', '.ogg'))]

if not violent_files or not non_violent_files:
    raise ValueError("Make sure both folders have audio files!")

rand_violent = random.choice(violent_files)
rand_non_violent = random.choice(non_violent_files)

print(f"🎧 Violent sample: {rand_violent}")
display(Audio(os.path.join(VIOLENT_DIR, rand_violent)))

print(f"\n🎵 Non-Violent sample: {rand_non_violent}")
display(Audio(os.path.join(NON_VIOLENT_DIR, rand_non_violent)))


In [ ]:
import os

# === CONFIG ===
VIOLENT_DIR = "/content/drive/MyDrive/split_audios/Violence"       # <-- change to your actual folder
NON_VIOLENT_DIR = "/content/drive/MyDrive/split_audios/NonViolence" # <-- change to your actual folder


# === COUNT FILES ===
violent_files = [f for f in os.listdir(VIOLENT_DIR) if f.endswith(('.wav', '.mp3', '.ogg'))]
non_violent_files = [f for f in os.listdir(NON_VIOLENT_DIR) if f.endswith(('.wav', '.mp3', '.ogg'))]

violent_count = len(violent_files)
non_violent_count = len(non_violent_files)

print(f"Total Violent samples: {violent_count}")
print(f"Total Non-Violent samples: {non_violent_count}")

In [ ]:
pip install audiomentations pydub scikit-learn

In [ ]:
import os
import random
import shutil
import numpy as np
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift
from pydub import AudioSegment
from sklearn.model_selection import train_test_split

# === CONFIG ===
VIOLENT_DIR = "/content/drive/MyDrive/split_audios/Violence"
NON_VIOLENT_DIR = "/content/drive/MyDrive/split_audios/NonViolence"
OUTPUT_DIR = "/content/drive/MyDrive/processed_audios"

# Create output folders
for folder in ["train/Violence", "train/NonViolence", "test/Violence", "test/NonViolence"]:
    os.makedirs(os.path.join(OUTPUT_DIR, folder), exist_ok=True)

# === PARAMETERS ===
TARGET_COUNT = 235   # ✅ changed target per class to 235
TEST_SIZE = 0.2      # 80/20 split
RANDOM_SEED = 42
SHIFT_PROB = 0.5     # probability to apply manual time-shift
MAX_SHIFT_SECONDS = 0.5  # max shift in seconds (±)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# === AUGMENTATION PIPELINE (audiomentations) ===
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.85, max_rate=1.2, p=0.5),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
])

def manual_time_shift(samples: np.ndarray, sample_rate: int, max_shift_seconds: float):
    """Shift audio by rolling the array within ±max_shift_seconds."""
    if max_shift_seconds <= 0:
        return samples
    max_shift_samples = int(max_shift_seconds * sample_rate)
    if max_shift_samples <= 0:
        return samples
    shift = random.randint(-max_shift_samples, max_shift_samples)
    return np.roll(samples, shift)

def augment_class(class_name, class_dir, target_count):
    """Augment a class directory up to target_count files"""
    files = [os.path.join(class_dir, f) for f in os.listdir(class_dir)
             if f.lower().endswith(('.wav', '.mp3', '.ogg', '.flac'))]
    print(f"\n🎧 {class_name}: {len(files)} original samples")

    if len(files) < target_count:
        needed = target_count - len(files)
        augmented_files = []
        for i in range(needed):
            src_file = random.choice(files)
            audio = AudioSegment.from_file(src_file)
            # convert to mono for simplicity
            if audio.channels > 1:
                audio = audio.set_channels(1)

            samples = np.array(audio.get_array_of_samples()).astype(np.float32)
            sample_rate = audio.frame_rate

            # Normalize to [-1, 1]
            max_val = np.max(np.abs(samples)) + 1e-9
            samples = samples / max_val

            # Apply audiomentations
            augmented = augment(samples=samples, sample_rate=sample_rate)

            # Optionally apply manual time shift
            if random.random() < SHIFT_PROB:
                augmented = manual_time_shift(augmented, sample_rate, MAX_SHIFT_SECONDS)

            # Convert back to int16
            augmented = np.clip(augmented, -1.0, 1.0)
            augmented_int16 = (augmented * 32767).astype(np.int16)

            # Save augmented file
            augmented_audio = AudioSegment(
                augmented_int16.tobytes(),
                frame_rate=sample_rate,
                sample_width=2,
                channels=1
            )

            base_name = os.path.splitext(os.path.basename(src_file))[0]
            aug_name = f"aug_{i}_{base_name}.wav"
            aug_path = os.path.join(class_dir, aug_name)
            augmented_audio.export(aug_path, format="wav")
            augmented_files.append(aug_path)

            if (i + 1) % 20 == 0:
                print(f"  -> created {i+1}/{needed} augmented samples for {class_name}")

        files += augmented_files

    print(f"✅ After augmentation: {len(files)} total samples for {class_name}")
    return files

# === RUN AUGMENTATION FOR BOTH CLASSES ===
violent_files = augment_class("Violence", VIOLENT_DIR, TARGET_COUNT)
non_violent_files = augment_class("NonViolence", NON_VIOLENT_DIR, TARGET_COUNT)

# === SPLIT DATA ===
v_train, v_test = train_test_split(violent_files, test_size=TEST_SIZE, random_state=RANDOM_SEED)
nv_train, nv_test = train_test_split(non_violent_files, test_size=TEST_SIZE, random_state=RANDOM_SEED)

# === COPY FILES TO DESTINATION ===
def copy_files(file_list, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
    for f in file_list:
        shutil.copy(f, dest_folder)

copy_files(v_train, os.path.join(OUTPUT_DIR, "train/Violence"))
copy_files(v_test, os.path.join(OUTPUT_DIR, "test/Violence"))
copy_files(nv_train, os.path.join(OUTPUT_DIR, "train/NonViolence"))
copy_files(nv_test, os.path.join(OUTPUT_DIR, "test/NonViolence"))

# === DONE ===
print("\n✅ Data augmentation and processing complete!")
print(f"Train/Test split = {1 - TEST_SIZE:.0%}/{TEST_SIZE:.0%}")
print(f"Training Violent: {len(v_train)} | Testing Violent: {len(v_test)}")
print(f"Training Non-Violent: {len(nv_train)} | Testing Non-Violent: {len(nv_test)}")
print(f"Files saved to: {OUTPUT_DIR}")

In [ ]:
import os
import random
import shutil
import numpy as np
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift
from pydub import AudioSegment
from sklearn.model_selection import train_test_split

# === CONFIG ===
VIOLENT_DIR = "/content/drive/MyDrive/split_audios/Violence"
NON_VIOLENT_DIR = "/content/drive/MyDrive/split_audios/NonViolence"
OUTPUT_DIR = "/content/drive/MyDrive/processed_audios"

# Create output folders
for folder in ["train/Violence", "train/NonViolence", "test/Violence", "test/NonViolence"]:
    os.makedirs(os.path.join(OUTPUT_DIR, folder), exist_ok=True)

# === PARAMETERS ===
TARGET_COUNT = 255   # target count per class
TEST_SIZE = 0.2      # 80/20 split
RANDOM_SEED = 42
SHIFT_PROB = 0.5     # probability to apply manual time-shift
MAX_SHIFT_SECONDS = 0.5  # max shift in seconds (±)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# === AUGMENTATION PIPELINE (audiomentations) ===
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.85, max_rate=1.2, p=0.5),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
])

def manual_time_shift(samples: np.ndarray, sample_rate: int, max_shift_seconds: float):
    """
    Shift audio by rolling the array. Shift amount chosen uniformly from
    [-max_shift_seconds, +max_shift_seconds].
    """
    if max_shift_seconds <= 0:
        return samples
    max_shift_samples = int(max_shift_seconds * sample_rate)
    if max_shift_samples <= 0:
        return samples
    shift = random.randint(-max_shift_samples, max_shift_samples)
    return np.roll(samples, shift)

def augment_class(class_name, class_dir, target_count):
    """Augment a class directory up to target_count files"""
    files = [os.path.join(class_dir, f) for f in os.listdir(class_dir)
             if f.lower().endswith(('.wav', '.mp3', '.ogg', '.flac'))]
    print(f"\n🎧 {class_name}: {len(files)} original samples")

    if len(files) < target_count:
        needed = target_count - len(files)
        augmented_files = []
        for i in range(needed):
            src_file = random.choice(files)
            audio = AudioSegment.from_file(src_file)
            # convert to mono for simplicity (avoids channel interleaving issues)
            if audio.channels > 1:
                audio = audio.set_channels(1)

            samples = np.array(audio.get_array_of_samples()).astype(np.float32)
            sample_rate = audio.frame_rate

            # Normalize to [-1, 1]
            max_val = np.max(np.abs(samples)) + 1e-9
            samples = samples / max_val

            # audiomentations expects float32 in [-1,1]
            augmented = augment(samples=samples, sample_rate=sample_rate)

            # optionally apply manual time shift
            if random.random() < SHIFT_PROB:
                augmented = manual_time_shift(augmented, sample_rate, MAX_SHIFT_SECONDS)

            # clip and convert to int16
            augmented = np.clip(augmented, -1.0, 1.0)
            augmented_int16 = (augmented * 32767).astype(np.int16)

            # Convert back to AudioSegment and export
            augmented_audio = AudioSegment(
                augmented_int16.tobytes(),
                frame_rate=sample_rate,
                sample_width=2,   # int16 -> 2 bytes
                channels=1
            )

            # unique name to avoid collisions
            base_name = os.path.splitext(os.path.basename(src_file))[0]
            aug_name = f"aug_{i}_{base_name}.wav"
            aug_path = os.path.join(class_dir, aug_name)
            augmented_audio.export(aug_path, format="wav")
            augmented_files.append(aug_path)

            if (i + 1) % 20 == 0:
                print(f"  -> created {i+1}/{needed} augmented samples for {class_name}")

        files += augmented_files

    print(f"✅ After augmentation: {len(files)} total samples for {class_name}")
    return files

# === RUN AUGMENTATION FOR BOTH CLASSES ===
violent_files = augment_class("Violence", VIOLENT_DIR, TARGET_COUNT)
non_violent_files = augment_class("NonViolence", NON_VIOLENT_DIR, TARGET_COUNT)

# === SPLIT DATA ===
v_train, v_test = train_test_split(violent_files, test_size=TEST_SIZE, random_state=RANDOM_SEED)
nv_train, nv_test = train_test_split(non_violent_files, test_size=TEST_SIZE, random_state=RANDOM_SEED)

# === COPY FILES TO DESTINATION ===
def copy_files(file_list, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)
    for f in file_list:
        shutil.copy(f, dest_folder)

copy_files(v_train, os.path.join(OUTPUT_DIR, "train/Violence"))
copy_files(v_test, os.path.join(OUTPUT_DIR, "test/Violence"))
copy_files(nv_train, os.path.join(OUTPUT_DIR, "train/NonViolence"))
copy_files(nv_test, os.path.join(OUTPUT_DIR, "test/NonViolence"))

# === DONE ===
print("\n✅ Data augmentation and processing complete!")
print(f"Train/Test split = {1 - TEST_SIZE:.0%}/{TEST_SIZE:.0%}")
print(f"Training Violent: {len(v_train)} | Testing Violent: {len(v_test)}")
print(f"Training Non-Violent: {len(nv_train)} | Testing Non-Violent: {len(nv_test)}")
print(f"Files saved to: {OUTPUT_DIR}")


In [ ]:
# === Install required packages ===
!pip install torch torchaudio torchvision --quiet
!pip install transformers datasets librosa soundfile tqdm --quiet
!pip install accelerate evaluate --quiet

In [ ]:
from transformers import AutoProcessor, AutoModelForAudioClassification

# Example: use Audio Spectrogram Transformer (AST)
model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"

processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)

print("✅ Transformer model loaded:", model_name)

In [ ]:
import os
import librosa
import torch
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import AutoProcessor, AutoModelForAudioClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
from transformers import AutoProcessor, AutoModelForAudioClassification
import torch
import torch.nn as nn

model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"

# Load pretrained processor
processor = AutoProcessor.from_pretrained(model_name)

# Load pretrained model first
model = AutoModelForAudioClassification.from_pretrained(model_name)

# Get hidden size from config (used for classifier input)
hidden_size = model.classifier.dense.in_features

# Replace classifier with new 2-class head
model.classifier = nn.Sequential(
    nn.Linear(hidden_size, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 2)
)

# Update model config for 2 labels
label2id = {"NonViolence": 0, "Violence": 1}
id2label = {v: k for k, v in label2id.items()}
model.config.num_labels = 2
model.config.label2id = label2id
model.config.id2label = id2label

print("✅ Replaced AST classifier head for 2-class fine-tuning.")
print(model.classifier)

In [ ]:
import os, librosa
from datasets import Dataset, DatasetDict

def load_audio_split(base_dir):
    data_train = {"audio": [], "label": []}
    data_test = {"audio": [], "label": []}

    # Loop through both class folders
    for label_name in ["NonViolence", "Violence"]:
        # Corrected paths
        train_dir = os.path.join(base_dir, "train", label_name)
        test_dir = os.path.join(base_dir, "test", label_name)

        # === TRAIN FILES ===
        if os.path.exists(train_dir):
            files = [f for f in os.listdir(train_dir) if f.endswith((".wav", ".mp3", ".ogg"))]
            # limit each class to 132 samples
            files = files[:132] if len(files) > 132 else files
            for f in files:
                data_train["audio"].append(os.path.join(train_dir, f))
                data_train["label"].append(model.config.label2id[label_name])

        # === TEST FILES ===
        if os.path.exists(test_dir):
            for f in os.listdir(test_dir):
                if f.endswith((".wav", ".mp3", ".ogg")):
                    data_test["audio"].append(os.path.join(test_dir, f))
                    data_test["label"].append(model.config.label2id[label_name])

    return data_train, data_test


# ✅ Correct base directory (not /train)
base_dir = "/content/drive/MyDrive/processed_audios"

train_data, test_data = load_audio_split(base_dir)

train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

print(f"✅ Loaded {len(train_dataset)} train and {len(test_dataset)} test samples.")


In [ ]:
import os, librosa
from datasets import Dataset, DatasetDict

def load_audio_split(base_dir):
    data_train = {"audio": [], "label": []}
    data_test = {"audio": [], "label": []}

    for label_name in ["NonViolence", "Violence"]:
        train_dir = os.path.join(base_dir, "train", label_name)
        test_dir = os.path.join(base_dir, "test", label_name)

        for f in os.listdir(train_dir):
            if f.endswith((".wav", ".mp3", ".ogg")):
                data_train["audio"].append(os.path.join(train_dir, f))
                data_train["label"].append(model.config.label2id[label_name])

        for f in os.listdir(test_dir):
            if f.endswith((".wav", ".mp3", ".ogg")):
                data_test["audio"].append(os.path.join(test_dir, f))
                data_test["label"].append(model.config.label2id[label_name])

    return data_train, data_test

base_dir = "/content/drive/MyDrive/processed_audios"
train_data, test_data = load_audio_split(base_dir)

train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

print(f"✅ Loaded {len(train_dataset)} train and {len(test_dataset)} test samples.")


In [ ]:
from transformers import ASTFeatureExtractor, AutoModelForAudioClassification
import torch, librosa, numpy as np
from datasets import Dataset, DatasetDict

# --- Load correct feature extractor and model ---
model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"
feature_extractor = ASTFeatureExtractor.from_pretrained(model_name)
model = AutoModelForAudioClassification.from_pretrained(model_name)


In [ ]:
from transformers import ASTFeatureExtractor, AutoModelForAudioClassification
import torch.nn as nn

# --- Find hidden size safely ---
try:
    hidden_size = model.classifier.dense.in_features  # if it's an ASTMLPHead
except AttributeError:
    # Fallback if classifier is already Sequential or changed structure
    hidden_size = model.classifier[0].in_features if isinstance(model.classifier, nn.Sequential) else model.classifier.in_features

# --- Replace classifier for 2 labels ---
model.classifier = nn.Sequential(
    nn.Linear(hidden_size, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, 2)
)

# --- Label mappings ---
label2id = {"NonViolence": 0, "Violence": 1}
id2label = {v: k for k, v in label2id.items()}
model.config.num_labels = 2
model.config.label2id = label2id
model.config.id2label = id2label

print("✅ Replaced classifier head with 2-class Sequential layer")
print(model.classifier)


In [ ]:
import librosa
import numpy as np
from datasets import load_dataset

# If you already created your dataset, skip this. Otherwise, recreate:
# dataset = load_dataset("path/to/your/train_test_data", split=["train", "test"])
# or, if you manually built it:
# dataset = DatasetDict({"train": train_ds, "test": test_ds})

def preprocess_function(batch):
    audios = []
    for path in batch["audio"]:
        # Load with original sampling rate
        arr, sr = librosa.load(path, sr=None, mono=True)
        # Resample if needed
        if sr != 16000:
            arr = librosa.resample(arr.astype(np.float32), orig_sr=sr, target_sr=16000)
        else:
            arr = arr.astype(np.float32)

        # Pad super short clips (AST needs window ≥400)
        if len(arr) < 400:
            arr = np.pad(arr, (0, 400 - len(arr)))
        audios.append(arr)

    inputs = feature_extractor(
        audios,
        sampling_rate=16000,
        padding="max_length",
        max_length=160000,   # ~10 s
        truncation=True,
        return_tensors=None
    )
    return {"input_values": inputs["input_values"], "label": batch["label"]}

dataset = dataset.map(preprocess_function, batched=True, batch_size=4)
dataset.set_format(type="torch", columns=["input_values", "label"])
print("✅ Dataset ready for training")


In [ ]:
from transformers import TrainingArguments, Trainer, set_seed
import numpy as np
import evaluate

set_seed(42)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./violence_audio_model",
    do_eval=True,
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=False,     # ✅ disabled for compatibility
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

print("✅ Trainer initialized successfully. Ready to train!")


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!pip uninstall -y wandb

In [ ]:
!pip install -q accelerate
from accelerate import Accelerator
Accelerator().free_memory()

import gc, torch
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import TrainingArguments, Trainer, set_seed
import numpy as np
import evaluate

set_seed(42)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./violence_audio_model",
    do_eval=False,
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=False,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

print("✅ Trainer reinitialized successfully. Ready to train!")


In [ ]:
trainer.train()

In [ ]:
# Evaluate the fine-tuned model
eval_results = trainer.evaluate()

# Print the evaluation metrics nicely
print("📊 Evaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

# Optional — just show accuracy clearly
print("\n✅ Model Accuracy:", eval_results["eval_accuracy"])

In [ ]:
save_dir = "/content/drive/MyDrive/audio_violence_detection_model"

# Save model and config
model.save_pretrained(save_dir)

# Also save the feature extractor (important for loading later)
feature_extractor.save_pretrained(save_dir)

print(f"✅ Audio model saved successfully at: {save_dir}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ✅ Get predictions on the test set
predictions = trainer.predict(dataset["test"])

# Extract predicted labels (highest probability)
y_pred = np.argmax(predictions.predictions, axis=-1)

# True labels
y_true = predictions.label_ids

# ✅ Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Define label names (order: [NonViolence, Violence])
labels = ["NonViolence", "Violence"]

# ✅ Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap="Blues", values_format='d')

plt.title("Confusion Matrix: Violence vs Non-Violence Audio")
plt.show()

In [ ]:
import torch
import librosa
import numpy as np

# 🎯 Give one audio path manually
audio_path = "/content/drive/MyDrive/processed_audios/test/Violence/angry_022_V0.wav"

# Label mapping — must match training order
id2label = {0: "NonViolence", 1: "Violence"}

# ✅ Function to predict one audio file
def predict_audio_label(audio_path):
    waveform, sr = librosa.load(audio_path, sr=16000)

    inputs = feature_extractor(
        waveform,
        sampling_rate=sr,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # Move to GPU if available
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        pred_id = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][pred_id].item()

    label = id2label[pred_id]
    return label, confidence

# ✅ Run prediction
label, confidence = predict_audio_label(audio_path)

print(f"🎧 Audio: {audio_path}")
print(f"→ Predicted Label: {label}")
print(f"→ Confidence: {confidence:.2f}")


In [ ]:
import torch
import librosa
import numpy as np

# 🎯 Give one audio path manually
audio_path = "/content/drive/MyDrive/processed_audios/test/NonViolence/angry_017_NV1.wav"

# Label mapping — must match training order
id2label = {0: "NonViolence", 1: "Violence"}

# ✅ Function to predict one audio file
def predict_audio_label(audio_path):
    waveform, sr = librosa.load(audio_path, sr=16000)

    inputs = feature_extractor(
        waveform,
        sampling_rate=sr,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # Move to GPU if available
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        pred_id = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][pred_id].item()

    label = id2label[pred_id]
    return label, confidence

# ✅ Run prediction
label, confidence = predict_audio_label(audio_path)

print(f"🎧 Audio: {audio_path}")
print(f"→ Predicted Label: {label}")
print(f"→ Confidence: {confidence:.2f}")


In [ ]:
import torch
import librosa
import numpy as np

# 🎯 Give one audio path manually
audio_path = "/content/drive/MyDrive/processed_audios/test/Violence/angry_034_V0.wav"

# Label mapping — must match training order
id2label = {0: "NonViolence", 1: "Violence"}

# ✅ Function to predict one audio file
def predict_audio_label(audio_path):
    waveform, sr = librosa.load(audio_path, sr=16000)

    inputs = feature_extractor(
        waveform,
        sampling_rate=sr,
        return_tensors="pt",
        padding=True,
        truncation=True
    )

    # Move to GPU if available
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        pred_id = torch.argmax(probs, dim=-1).item()
        confidence = probs[0][pred_id].item()

    label = id2label[pred_id]
    return label, confidence

# ✅ Run prediction
label, confidence = predict_audio_label(audio_path)

print(f"🎧 Audio: {audio_path}")
print(f"→ Predicted Label: {label}")
print(f"→ Confidence: {confidence:.2f}")


In [ ]:
import torch, librosa, numpy as np, subprocess, os, uuid
from transformers import (
    VideoMAEForVideoClassification,
    AutoModelForAudioClassification,
    ASTFeatureExtractor,
)
from decord import VideoReader, cpu
import torchvision.transforms as T
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import uuid
from IPython.display import HTML, Audio, display
from base64 import b64encode


In [ ]:
VIDEO_MODEL_PATH = "/content/drive/MyDrive/video_violence_detection_model"   # saved with save_pretrained()
AUDIO_MODEL_PATH = "/content/drive/MyDrive/audio_violence_detection_model"   # saved with save_pretrained()
TEST_DIR = "/content/violence_dataset_split/test"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {DEVICE}")

In [ ]:
video_model = VideoMAEForVideoClassification.from_pretrained(VIDEO_MODEL_PATH)
video_model.to(DEVICE)
video_model.eval()
print(" Video model loaded successfully")

In [ ]:
feature_extractor = ASTFeatureExtractor.from_pretrained(AUDIO_MODEL_PATH)
audio_model = AutoModelForAudioClassification.from_pretrained(AUDIO_MODEL_PATH)
audio_model.to(DEVICE)
audio_model.eval()
print("Audio model loaded successfully")

# Label mapping (must match training)
id2label = {0: "NonViolence", 1: "Violence"}
label2id = {"NonViolence": 0, "Violence": 1}

In [ ]:
def has_audio(video_path):
    """Check if the video has an audio stream."""
    cmd = [
        "ffprobe", "-v", "error", "-select_streams", "a",
        "-show_entries", "stream=codec_type", "-of", "csv=p=0", video_path
    ]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    return bool(result.stdout.strip())

def extract_audio_from_video(video_path):
    """Extracts audio if present."""
    temp_audio_path = f"/content/temp_audio_{uuid.uuid4().hex}.wav"
    command = [
        "ffmpeg", "-i", video_path, "-q:a", "0", "-map", "a",
        temp_audio_path, "-y", "-loglevel", "error"
    ]
    subprocess.run(command)
    return temp_audio_path if os.path.exists(temp_audio_path) else None


In [ ]:
def load_video_frames(video_path, num_frames=16, frame_size=(224,224)):
    vr = VideoReader(video_path, ctx=cpu(0))
    total = len(vr)
    frame_idx = torch.linspace(0, total-1, num_frames).long()
    frames = vr.get_batch(frame_idx).float()/255.0
    frames = frames.permute(0,3,1,2)
    transform = T.Compose([
        T.Resize(frame_size),
        T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
    ])
    frames = torch.stack([transform(f) for f in frames])
    return frames.unsqueeze(0)

In [ ]:
def load_audio_features(audio_path):
    waveform, sr = librosa.load(audio_path, sr=16000)
    return feature_extractor(
        waveform, sampling_rate=sr,
        return_tensors="pt", padding=True, truncation=True
    )


In [ ]:
def predict_fusion(video_path, alpha=0.6):
    video_frames = load_video_frames(video_path).to(DEVICE)
    with torch.no_grad():
        video_logits = video_model(video_frames).logits

    if has_audio(video_path):
        audio_path = extract_audio_from_video(video_path)
        if audio_path:
            audio_inputs = load_audio_features(audio_path)
            audio_inputs = {k:v.to(DEVICE) for k,v in audio_inputs.items()}
            with torch.no_grad():
                audio_logits = audio_model(**audio_inputs).logits
            os.remove(audio_path)
            fused_logits = alpha*video_logits + (1-alpha)*audio_logits
            probs = torch.nn.functional.softmax(fused_logits, dim=-1)
            return torch.argmax(probs, dim=-1).item()
    # fallback: no audio
    probs = torch.nn.functional.softmax(video_logits, dim=-1)
    return torch.argmax(probs, dim=-1).item()

In [ ]:
true_labels, pred_labels = [], []

for cls in ["NonViolence","Violence"]:
    folder = os.path.join(TEST_DIR, cls)
    label = label2id[cls]
    for file in os.listdir(folder):
        if file.lower().endswith((".mp4",".avi",".mov")):
            path = os.path.join(folder,file)
            try:
                pred = predict_fusion(path, alpha=0.6)
                true_labels.append(label)
                pred_labels.append(pred)
                print(f"{file}: True={cls}, Pred={id2label[pred]}")
            except Exception as e:
                print(f"⚠️ Error processing {file}: {e}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc, classification_report
)
import pandas as pd

In [ ]:
accuracy = accuracy_score(true_labels, pred_labels)
precision = precision_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)

print(" Evaluation Metrics:")
print(f"Accuracy : {accuracy*100:.2f}%")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1 Score : {f1:.3f}")

In [ ]:
print("\nDetailed Classification Report:")
print(classification_report(true_labels, pred_labels, target_names=["NonViolence", "Violence"]))

In [ ]:
cm = confusion_matrix(true_labels, pred_labels)
labels = ["NonViolence", "Violence"]

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap="YlGnBu", xticklabels=labels, yticklabels=labels)
plt.title("Fusion Model Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:
data = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1-Score"],
    "Fusion Model": [f"{accuracy*100:.2f}%", f"{precision:.3f}", f"{recall:.3f}", f"{f1:.3f}"]
}
metrics_df = pd.DataFrame(data)
display(metrics_df.style.set_caption("Fusion Model Performance Summary"))


In [ ]:
correct = np.sum(np.array(true_labels) == np.array(pred_labels))
incorrect = len(true_labels) - correct

plt.figure(figsize=(5,5))
plt.pie(
    [correct, incorrect],
    labels=["Correct Predictions", "Incorrect Predictions"],
    autopct='%1.1f%%', startangle=140,
    colors=["#4CAF50", "#F44336"]
)
plt.title("Fusion Model Prediction Distribution")
plt.show()

In [ ]:
def display_video(video_path, width=480):
    """Display video inline in Colab."""
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return f"""
        <video width={width} controls>
            <source src="{data_url}" type="video/mp4">
        </video>
    """

In [ ]:
def predict_single_clip(video_path, alpha=0.6):
    print(f" Processing: {video_path}\n")

    # Display video
    display(HTML(display_video(video_path, width=480)))

    # ---- Video Prediction ----
    video_frames = load_video_frames(video_path).to(DEVICE)
    with torch.no_grad():
        video_logits = video_model(video_frames).logits

    # ---- Audio Handling ----
    fused_logits = video_logits
    audio_available = has_audio(video_path)
    if audio_available:
        audio_path = extract_audio_from_video(video_path)
        if audio_path:
            display(Audio(audio_path, autoplay=False))
            audio_inputs = load_audio_features(audio_path)
            audio_inputs = {k: v.to(DEVICE) for k, v in audio_inputs.items()}
            with torch.no_grad():
                audio_logits = audio_model(**audio_inputs).logits
            # Weighted fusion
            fused_logits = alpha * video_logits + (1 - alpha) * audio_logits
            os.remove(audio_path)
        else:
            print(" Audio extraction failed — using video model only.")
    else:
        print(" No audio stream detected — using video model only.")

    # ---- Prediction ----
    probs = torch.nn.functional.softmax(fused_logits, dim=-1)
    pred_id = torch.argmax(probs, dim=-1).item()
    confidence = probs[0, pred_id].item()
    label = id2label[pred_id]

    print(f"\n Predicted Label: {label}")
    print(f" Confidence: {confidence:.2f}")
    if audio_available:
        print(f" Video Weight: {alpha},  Audio Weight: {1 - alpha}")

    return label, confidence

In [ ]:
video_clip_path = "/content/drive/MyDrive/Colab Notebooks/Non_Violence1.mp4"  # 👈 your clip (with audio + video)
label, conf = predict_single_clip(video_clip_path, alpha=0.6)

In [ ]:
video_clip_path = "/content/drive/MyDrive/Colab Notebooks/Violence1.mp4" # 👈 your clip (with audio + video)
label, conf = predict_single_clip(video_clip_path, alpha=0.6)

In [ ]:
video_clip_path = "/content/drive/MyDrive/Colab Notebooks/Non_Violence2.mp4" # 👈 your clip (with audio + video)
label, conf = predict_single_clip(video_clip_path, alpha=0.6)

In [ ]:
video_clip_path = "/content/drive/MyDrive/Colab Notebooks/Violence2.mp4" # 👈 your clip (with audio + video)
label, conf = predict_single_clip(video_clip_path, alpha=0.6)